In [1]:
# OpenAI API, RAG, and Agent

import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)


In [3]:
## Example 1: Plain OpenAI API Request
def plain_openai_request(prompt):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o",
    )
    return chat_completion.choices[0].message.content

In [4]:
# Example prompt
plain_prompt = "Who won 2024 US election?"
plain_result = plain_openai_request(plain_prompt)
print(f"Plain API Request Result: {plain_result}")

Plain API Request Result: I'm unable to provide real-time information, including the results of the 2024 US election, as my training only goes up to October 2023. For the most current information, please check reputable news sources or official announcements.


In [5]:
## Example 2: Retrieval-Augmented Generation (RAG)
def retrieve_information(query):
    knowledge_base = {
        "Who won 2024 US election?":
        """
        Presidential elections were held in the United States on November 5, 2024. 
        The Republican Party's ticket—Donald Trump, who was the 45th president of 
        the United States from 2017 to 2021, and JD Vance, the junior U.S. senator 
        from Ohio—defeated the Democratic Party's ticket—Kamala Harris, the 
        incumbent vice president, and Tim Walz, the 41st governor of Minnesota.
        Trump and Vance were inaugurated as the 47th president and the 50th vice 
        president on January 20, 2025.
        """
    }
    return knowledge_base.get(query, "No relevant information found.")

def rag_request(query):
    retrieved_info = retrieve_information(query)
    prompt = f"Based on the retrieved information: '{retrieved_info}', answer the following question: {query}"
    return plain_openai_request(prompt)

In [6]:
# Example RAG query
rag_query = "Who won 2024 US election?"
rag_result = rag_request(rag_query)
print(f"RAG Result: {rag_result}")

RAG Result: According to the retrieved information, Donald Trump and JD Vance won the 2024 US presidential election.


In [10]:
## Example 3: Agent Example
import warnings
warnings.filterwarnings('ignore')
from crewai import Agent, Crew, Task
from crewai_tools import (
  ScrapeWebsiteTool,
  SerperDevTool
)


os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    tools = [scrape_tool, search_tool],
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True
)


writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    tools = [scrape_tool, search_tool],
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)


plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

2025-01-27 00:36:25,411 - 140704719306176 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


LLM value is None
LLM value is None
LLM value is None


In [11]:
topic = "Who won 2024 US election?"
result = crew.kickoff(inputs={"topic": topic})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Who won 2024 US election?.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Thought: I need to gather the latest information regarding the 2024 US election results, including trends, key players, and noteworthy news. This will help me develop an informed content plan for the blog article.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Who won 2024 US election?\"}"
## Tool Output: 
{'searchParameters': {'q': 'Who won 2024 US election?', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '2024 United States presidential election - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/2024_United_States_presidential_election

In [14]:
from IPython.display import Markdown

Markdown(result.raw)

```markdown
# Who Won the 2024 US Election?

## Introduction

The 2024 US presidential election has come to a close, marking a significant moment in American political history. This election not only reflected the hopes and concerns of millions of voters but also set the stage for the future political landscape. The main candidates, Donald Trump, representing the Republican Party, and Kamala Harris, the Democratic incumbent, have both been pivotal figures in shaping the current political discourse.

Ultimately, the winner of the 2024 US election was Donald Trump, marking a remarkable comeback for the former president. As he transitions back into the role of commander-in-chief, the implications of his victory will undoubtedly reverberate across various sectors of American life, influencing policy direction and voter sentiments in the years to come.

## Election Results Overview

In this election, Donald Trump secured a decisive victory with **312 electoral college votes** compared to Kamala Harris's **226 electoral college votes**. The electoral college is pivotal in determining the outcome of the presidential race, as candidates must reach a majority of the 538 electoral votes available. Trump's successful campaign strategy resonated with voters in key regions, allowing him to surpass the necessary threshold tied to this system.

In terms of the popular vote, Trump garnered **77,302,416 votes**, accounting for **49.8%**, while Harris received **75,012,178 votes**, which amounts to **48.3%**. These numbers underscore a closely contested race, but with Trump leading in both the electoral college and popular vote, he emerged victorious. States such as Florida and North Carolina played critical roles in shifting towards Trump, ultimately aiding his electoral college success.

## Key Trends and Insights

Analyzing the voting trends and demographic shifts in the 2024 election unveils insights about the evolving political landscape of the United States. Exit polls indicated a notable increase in support for Trump among Hispanic voters, a demographic that had previously leaned more toward Democrats in prior elections. Furthermore, suburban areas showed significant shifts, reflecting a reconfiguration of traditional voting patterns that could shape future contests.

Key issues such as the economy, healthcare, and immigration played a pivotal role in shaping voter decisions during this election cycle. Many voters identified economic recovery and job growth as their primary concerns, prompting them to back Trump's policies. In contrast, Harris's campaign focused on social justice and climate action, appealing to a more progressive base but ultimately lacking the widespread support necessary to secure re-election.

## Major Players and Their Political Implications

Trump's victory signifies a continuation of several policies from his previous administration, including tax reforms and deregulation efforts. As he reassumes the presidency, Trump's positions on issues like healthcare and immigration are expected to shape ongoing legislative priorities. His ability to navigate a potentially divided Congress will be crucial in advancing his agenda and fulfilling the promises made during his campaign.

Conversely, Kamala Harris's loss raises important questions about the future direction of the Democratic Party. Her leadership during the campaign emphasized unity and progressive reforms, yet her inability to secure re-election may lead to a critical re-evaluation of strategies within the party as they prepare for upcoming contests. Potential shifts in both policies and leadership roles will be essential for Democrats seeking to regain a foothold in key battleground states.

## Conclusion

The results of the 2024 US presidential election hold substantial significance for American democracy. Trump's return is not just a victory for the Republican Party; it also signals the continuation of divisive rhetoric and policies within the political arena. As the nation navigates these challenging times, active engagement with political developments remains vital for voters across the spectrum.

## Call to Action

We invite you to share your thoughts on the 2024 election results in the comments below. Engaging in discourse around this pivotal moment in history is crucial, and staying informed through reputable news sources will enhance our collective understanding of the dynamic political landscape we face.
```